In [ ]:
!pip install pydicom 
!pip install pylibjpeg[all]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import glob
import math
import six
from __future__ import (
    absolute_import,
    division,
    print_function,
    unicode_literals
)
from datetime import datetime
from math import ceil

import nibabel as nib
import numpy as np
import pandas as pd
import pydicom
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    BatchNormalization
)
from keras.layers.convolutional import (
    Conv3D,
    AveragePooling3D,
    MaxPooling3D
)
from keras.layers.merge import add
from keras.regularizers import l2
from keras import backend as K
from pydicom.pixel_data_handlers.util import apply_modality_lut
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence

In [13]:
drive_data_path = "/content/drive/My Drive/Medical Research/Data_Tümü/"
drive_patient_list_path = "/content/drive/My Drive/Medical Research/Hasta_Listesi.xlsx"
drive_log_path = "/content/drive/My Drive/Medical Research/logs"

In [14]:
def _bn_relu(input):
    """Helper to build a BN -> relu block (by @raghakot)."""
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu3D(**conv_params):
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1, 1))
    kernel_initializer = conv_params.setdefault(
        "kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer",
                                                l2(1e-4))

    def f(input):
        conv = Conv3D(filters=filters, kernel_size=kernel_size,
                      strides=strides, kernel_initializer=kernel_initializer,
                      padding=padding,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv3d(**conv_params):
    """Helper to build a  BN -> relu -> conv3d block."""
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer",
                                                "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer",
                                                l2(1e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv3D(filters=filters, kernel_size=kernel_size,
                      strides=strides, kernel_initializer=kernel_initializer,
                      padding=padding,
                      kernel_regularizer=kernel_regularizer)(activation)
    return f


def _shortcut3d(input, residual):
    """3D shortcut to match input and residual and merges them with "sum"."""
    stride_dim1 = ceil(input.shape[DIM1_AXIS] \
        / residual.shape[DIM1_AXIS])
    stride_dim2 = ceil(input.shape[DIM2_AXIS] \
        / residual.shape[DIM2_AXIS])
    stride_dim3 = ceil(input.shape[DIM3_AXIS] \
        / residual.shape[DIM3_AXIS])
    equal_channels = residual.shape[CHANNEL_AXIS] \
        == input.shape[CHANNEL_AXIS]

    shortcut = input
    if stride_dim1 > 1 or stride_dim2 > 1 or stride_dim3 > 1 \
            or not equal_channels:
        shortcut = Conv3D(
            filters=residual.shape[CHANNEL_AXIS],
            kernel_size=(1, 1, 1),
            strides=(stride_dim1, stride_dim2, stride_dim3),
            kernel_initializer="he_normal", padding="valid",
            kernel_regularizer=l2(1e-4)
            )(input)
    return add([shortcut, residual])


def _residual_block3d(block_function, filters, kernel_regularizer, repetitions,
                      is_first_layer=False):
    def f(input):
        for i in range(repetitions):
            strides = (1, 1, 1)
            if i == 0 and not is_first_layer:
                strides = (2, 2, 2)
            input = block_function(filters=filters, strides=strides,
                                   kernel_regularizer=kernel_regularizer,
                                   is_first_block_of_first_layer=(
                                       is_first_layer and i == 0)
                                   )(input)
        return input

    return f


def basic_block(filters, strides=(1, 1, 1), kernel_regularizer=l2(1e-4),
                is_first_block_of_first_layer=False):
    """Basic 3 X 3 X 3 convolution blocks. Extended from raghakot's 2D impl."""
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv3D(filters=filters, kernel_size=(3, 3, 3),
                           strides=strides, padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(input)
        else:
            conv1 = _bn_relu_conv3d(filters=filters,
                                    kernel_size=(3, 3, 3),
                                    strides=strides,
                                    kernel_regularizer=kernel_regularizer
                                    )(input)

        residual = _bn_relu_conv3d(filters=filters, kernel_size=(3, 3, 3),
                                   kernel_regularizer=kernel_regularizer
                                   )(conv1)
        return _shortcut3d(input, residual)

    return f


def bottleneck(filters, strides=(1, 1, 1), kernel_regularizer=l2(1e-4),
               is_first_block_of_first_layer=False):
    """Basic 3 X 3 X 3 convolution blocks. Extended from raghakot's 2D impl."""
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv3D(filters=filters, kernel_size=(1, 1, 1),
                              strides=strides, padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=kernel_regularizer
                              )(input)
        else:
            conv_1_1 = _bn_relu_conv3d(filters=filters, kernel_size=(1, 1, 1),
                                       strides=strides,
                                       kernel_regularizer=kernel_regularizer
                                       )(input)

        conv_3_3 = _bn_relu_conv3d(filters=filters, kernel_size=(3, 3, 3),
                                   kernel_regularizer=kernel_regularizer
                                   )(conv_1_1)
        residual = _bn_relu_conv3d(filters=filters * 4, kernel_size=(1, 1, 1),
                                   kernel_regularizer=kernel_regularizer
                                   )(conv_3_3)

        return _shortcut3d(input, residual)

    return f


def _handle_data_format():
    global DIM1_AXIS
    global DIM2_AXIS
    global DIM3_AXIS
    global CHANNEL_AXIS
    if K.image_data_format() == 'channels_last':
        DIM1_AXIS = 1
        DIM2_AXIS = 2
        DIM3_AXIS = 3
        CHANNEL_AXIS = 4
    else:
        CHANNEL_AXIS = 1
        DIM1_AXIS = 2
        DIM2_AXIS = 3
        DIM3_AXIS = 4


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class Resnet3DBuilder(object):
    """ResNet3D."""

    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions, reg_factor):
        """Instantiate a vanilla ResNet3D keras model.
        # Arguments
            input_shape: Tuple of input shape in the format
            (conv_dim1, conv_dim2, conv_dim3, channels) if dim_ordering='tf'
            (filter, conv_dim1, conv_dim2, conv_dim3) if dim_ordering='th'
            num_outputs: The number of outputs at the final softmax layer
            block_fn: Unit block to use {'basic_block', 'bottlenack_block'}
            repetitions: Repetitions of unit blocks
        # Returns
            model: a 3D ResNet model that takes a 5D tensor (volumetric images
            in batch) as input and returns a 1D vector (prediction) as output.
        """
        _handle_data_format()
        if len(input_shape) != 4:
            raise ValueError("Input shape should be a tuple "
                             "(conv_dim1, conv_dim2, conv_dim3, channels) "
                             "for tensorflow as backend or "
                             "(channels, conv_dim1, conv_dim2, conv_dim3) "
                             "for theano as backend")

        block_fn = _get_block(block_fn)
        input = Input(shape=input_shape)
        # first conv
        conv1 = _conv_bn_relu3D(filters=64, kernel_size=(7, 7, 7),
                                strides=(2, 2, 2),
                                kernel_regularizer=l2(reg_factor)
                                )(input)
        pool1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2),
                             padding="same")(conv1)

        # repeat blocks
        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block3d(block_fn, filters=filters,
                                      kernel_regularizer=l2(reg_factor),
                                      repetitions=r, is_first_layer=(i == 0)
                                      )(block)
            filters *= 2

        # last activation
        block_output = _bn_relu(block)

        # average poll and classification
        pool2 = AveragePooling3D(pool_size=(block.shape[DIM1_AXIS],
                                            block.shape[DIM2_AXIS],
                                            block.shape[DIM3_AXIS]),
                                 strides=(1, 1, 1))(block_output)
        flatten1 = Flatten()(pool2)
        if num_outputs > 1:
            dense = Dense(units=num_outputs,
                          kernel_initializer="he_normal",
                          activation="softmax",
                          kernel_regularizer=l2(reg_factor))(flatten1)
        else:
            dense = Dense(units=num_outputs,
                          kernel_initializer="he_normal",
                          activation="sigmoid",
                          kernel_regularizer=l2(reg_factor))(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs, reg_factor=1e-4):
        """Build resnet 18."""
        return Resnet3DBuilder.build(input_shape, num_outputs, basic_block,
                                     [2, 2, 2, 2], reg_factor=reg_factor)


In [15]:
def get_label(name, type, path):
    df = pd.read_excel(path)
    df["Name"] = df["Name"].str.lower()
    df = df.loc[df['Name'] == name]
    
    l1 = df[["2R"]].iloc[0].to_numpy()
    l2 = df[["4R"]].iloc[0].to_numpy()
    l3 = df[[7]].iloc[0].to_numpy()
    l4 = df[["2L"]].iloc[0].to_numpy()
    l5 = df[["4L"]].iloc[0].to_numpy()

    df = l1 or l2 or l3 or l4 or l5

    if(type == "train"):
        if df == 0:
            return np.array([1,0])
        else:
            return np.array([0,1])
    if(type == "predict"):
        return df


def get_names(path):
    
    df = pd.read_excel(path)
    df["Name"] = df["Name"].str.lower()

    df = df[["Name", "Uygunluk"]]
    df = df[df.Uygunluk == 1]
    df = df.dropna()

    df = df["Name"]
    df = df.to_numpy()

    pos = []
    neg = []

    for name in df:
        label = get_label(name, "predict", path)
        if(label == 1):
            pos.append(name)
        else:
            neg.append(name)

    return pos, neg

In [16]:
DATA_MIN = -140
DATA_MAX = 260

def cut(data, mid, size):
    mid_x, mid_y, mid_z = mid
    min_x = max(0, math.floor(mid_x - size // 2))
    max_x = min(math.ceil(mid_x + size // 2), data.shape[0]-1)
    min_y = max(0, math.floor(mid_y - size // 2))
    max_y = min(math.ceil(mid_y + size // 2), data.shape[1]-1)
    min_z = max(0, math.floor(mid_z - size // 2))
    max_z = min(math.ceil(mid_z + size // 2), data.shape[2]-1)
    data = data[min_x:max_x, min_y:max_y, min_z:max_z]
    return data

def normalize(data):
    data[data > DATA_MAX] = DATA_MAX
    data[data < DATA_MIN] = DATA_MIN
    data = (data - DATA_MIN) / (DATA_MAX - DATA_MIN)
    return data

def find_mid(data):
    non_zero = np.nonzero(data)
    x = non_zero[0]
    y = non_zero[1]
    z = non_zero[2]
    mid_z = (max(z) + min(z)) // 2
    mid_y = (max(y) + min(y)) // 2
    mid_x = (max(x) + min(x)) // 2
    return (mid_x, mid_y, mid_z)

def padding(data, size):
    x, y, z = data.shape
    img = np.zeros((size, size, size))
    x_diff = (size-x) // 2
    y_diff = (size-y) // 2
    z_diff = (size-z) // 2
    img[x_diff:x_diff+x, y_diff:y_diff+y, z_diff:z_diff+z] = data
    return img

def broadcast_to_shape(data, shape):
    x = data.shape[0] == shape
    y = data.shape[1] == shape
    z = data.shape[2] == shape
    if not x:
        diff = (shape - data.shape[0])
        data = np.pad(data, [(diff // 2, diff - (diff // 2)),(0,0),(0,0)], mode = "constant")
    if not y:
        diff = (shape - data.shape[1])
        data = np.pad(data, [(0,0),(diff // 2, diff - (diff // 2)),(0,0)], mode = "constant")
    if not x:
        diff = (shape - data.shape[2])
        data = np.pad(data, [(0,0),(0,0),(diff // 2, diff - (diff // 2))], mode = "constant")
    return data

def read_dicom(name, size, mid):
    path = drive_data_path + name + "/*.dcm"
    dcm_files = glob.glob(path)
    pixel_data = []
    data = []
    for dcm_file in dcm_files:
        dataset = pydicom.dcmread(dcm_file)
        data.append(dataset)
    slices = sorted(data, key=lambda s: s.ImagePositionPatient[2])
    for slice in slices:
        pixel_array = slice.pixel_array
        pixel_data.append(apply_modality_lut(pixel_array, slice))
    pixel_arr = np.asarray(pixel_data)
    dcm_data = cut(pixel_arr, mid, size).astype("float32")
    dcm_data = normalize(dcm_data).astype("float32")
    x = np.empty((size, size, size, 1))
    x[..., 0] = dcm_data
    return x

def read_data(name, size, segment, is_whole):
    nifti_path = drive_data_path + name + "/*.nii.gz"
    nifti_file = glob.glob(nifti_path)
    img = nib.load(nifti_file[0])
    data = img.get_fdata().transpose()
    if not is_whole:
        data[data != float(segment)] = 0.
    mid = find_mid(data)
    dcm = read_dicom(name, size, mid)
    data = cut(data, mid, size)
    return dcm, data

In [30]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, gen_type, name_list_path, segment=0.,is_whole=True, 
                 size = 100, batch_size=1, n_channels=1, shuffle=True):
        'Initialization'
        self.dim = (size,size,size)
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.size = size
        self.segment = segment
        self.is_whole = is_whole
        self.gen_type = gen_type
        self.name_list_path = name_list_path
        self.time = datetime.now().strftime("%m-%d-%Y %H-%M-%S")
       
   
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.list_IDs.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(self.list_IDs.shape[0] / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        list_IDs_temp = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
      
        X, y = self.__data_generation(list_IDs_temp)

        return X,y

    def __iter__(self):
        for item in (self[i] for i in range(len(self))):
            yield item
 
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size,2))
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ID = ID.lower()
            try:
                y[i] = get_label(ID,self.gen_type,self.name_list_path)
                X[i,], _ = read_data(ID,self.size,self.segment,self.is_whole)
            except Exception as e:
                file_name = drive_log_path + f"/log_{self.time}.txt"
                with open(file_name, 'a+') as f:
                    f.write(ID + "\n\n\n")
                    f.write(str(e))
                    f.write("\n---------------------------\n\n\n")

        return X,y

In [18]:
pos, neg = get_names(drive_patient_list_path)

In [21]:
neg = neg[:len(pos)]

In [22]:
train_names_pos, val_names_pos = tts(pos, test_size=0.2, random_state=42)
train_names_neg, val_names_neg = tts(neg, test_size=0.2, random_state=42)

In [23]:
train_names = np.asarray(train_names_pos + train_names_neg)
np.random.shuffle(train_names)
val_names = np.asarray(val_names_pos + val_names_neg)
np.random.shuffle(val_names)

In [31]:
train_gen = DataGenerator(train_names, "train", drive_patient_list_path, batch_size=1)
val_gen = DataGenerator(val_names, "train", drive_patient_list_path, batch_size=1)

In [25]:
model = Resnet3DBuilder.build_resnet_18((100, 100, 100, 1), 2)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])

In [26]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=drive_log_path)

In [ ]:
model.fit(train_gen, batch_size=1, epochs=50, validation_data=val_gen, callbacks = [tensorboard], verbose=2)

Epoch 1/50
